In [1]:
using BenchmarkTools
using Iterators: subsets

In [84]:
const words = lowercase.(strip.(split(readstring("data/113809of.fic"))));

In [85]:
import Collective

In [156]:
reload("Collective")

INFO: Recompiling stale cache file /Users/rdeits/.julia/lib/v0.5/Collective.ji for module Collective.


In [157]:
corpus = Collective.Corpus(words[1:Int(round(length(words) / 10000)):end]);

In [158]:
Collective.best_cluster(corpus, w, 3)

(String["fuzz","foo","lowered"],Collective.FeatureResult("has at least 1 reverse alphabetical bigrams",Collective.#49,Bool[false,false,true],5.903002375525849e-5))

In [147]:
Collective.best_feature(corpus, w)

Collective.FeatureResult("has at least 1 reverse alphabetical bigrams",Collective.#49,Bool[false,false,true,true,true],0.00019589391792554)

In [149]:
function cluster(corpus, words, n)
    vals = corpus.features.evaluate.(words)
    x = Inf
    best_subset = Int[]
    for subset in subsets(collect(1:length(vals)), n)
        f = Collective.best_feature(corpus, vals[subset])
        p = f.probability
        if p < x
            x = p
            best_subset = subset
        end
    end
    words[best_subset]
end

cluster (generic function with 1 method)

In [150]:
w = ["fuzz", "foo", "lowered", "levitate", "inanimate"]
@benchmark cluster($corpus, $w, 3)

BenchmarkTools.Trial: 
  memory estimate:  140.95 kb
  allocs estimate:  656
  --------------
  minimum time:     414.194 μs (0.00% GC)
  median time:      426.450 μs (0.00% GC)
  mean time:        487.894 μs (6.33% GC)
  maximum time:     8.838 ms (90.44% GC)
  --------------
  samples:          10000
  evals/sample:     1
  time tolerance:   5.00%
  memory tolerance: 1.00%

In [159]:
behave = shuffle([
    "annieproulx",
"commutative",
"hugoweaving",
"mountaindew",
"mozambique",
"sequoia",
"annotation",
"artificial",
"engineroom",
"individual",
"omnivorous",
"onlocation",
"almost",
"biopsy",
"chimp",
"films",
"ghost",
"tux",
"balked",
"barspoon",
"highnoon",
"klutzy",
"onyx",
"posted",
])

24-element Array{String,1}:
 "onyx"       
 "posted"     
 "tux"        
 "commutative"
 "engineroom" 
 "klutzy"     
 "onlocation" 
 "omnivorous" 
 "individual" 
 "mozambique" 
 "balked"     
 "ghost"      
 "hugoweaving"
 "annotation" 
 "almost"     
 "mountaindew"
 "highnoon"   
 "artificial" 
 "films"      
 "sequoia"    
 "chimp"      
 "biopsy"     
 "annieproulx"
 "barspoon"   

In [160]:
c1, f1 = Collective.best_cluster(corpus, behave, 6)
@show c1
sort(Collective.analyze(corpus, c1))[1]

c1 = String["tux","ghost","almost","films","chimp","biopsy"]


Collective.FeatureResult("has at least 1 reverse alphabetical bigrams",Collective.#49,Bool[false,false,false,false,false,false],7.720764319213579e-15)

In [162]:
remaining = setdiff(behave, c1)
c2, f2 = Collective.best_cluster(corpus, remaining, 6)
@show c2
sort(Collective.analyze(corpus, c2))[1]

c2 = String["commutative","mozambique","hugoweaving","mountaindew","sequoia","annieproulx"]


Collective.FeatureResult("has 5 unique vowels",Collective.#32,Bool[true,true,true,true,true,true],6.788029344188344e-13)

In [163]:
remaining = setdiff(setdiff(behave, c1), c2)
c3, f3 = Collective.best_cluster(corpus, remaining, 6)
@show c3
sort(Collective.analyze(corpus, c3))[1]

c3 = String["onyx","posted","klutzy","onlocation","balked","barspoon"]


Collective.FeatureResult("has at least 2 reverse sequential bigrams",Collective.#53,Bool[true,true,true,true,true,true],3.2390058912091412e-9)

In [164]:
remaining = setdiff(setdiff(setdiff(behave, c1), c2), c3)
@show remaining
Collective.best_feature(corpus, remaining)
# sort(Collective.analyze(corpus, remaining))[1]

remaining = String["engineroom","omnivorous","individual","annotation","highnoon","artificial"]


Collective.FeatureResult("has scrabble score 15",Collective.#28,Bool[false,true,true,false,true,true],0.0003434067294473176)